In [22]:
#@title

import warnings
warnings.filterwarnings('ignore')



import ipywidgets as widgets
from IPython.display import display, clear_output

In [23]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [24]:
# @title
#### PROFIL DE PUISSANCE ####

#Decollage
Pdec = 1;
Tdec = 1; #min

#Montée
Pmon = 0.75;
Tmon = 20; #min

#Croisière
Pcro = 0.53;
Tcro = 180; #min

#### INFO ATR 72 ####

Pmax = 4; #MW
Diam_int = 2; #m


#### INFO H2 ####

H_E_d = 120; #MJ/kg
H_cout = 4; #€
H_d = 71 #kg/m3

#### INFO TANK ####

wt = 0.25;
T_encomb = 0.8;

#### INFO VOL ####

Ticket_Cout = 100; #€ pour un vol de 3h


#### INFO FC ####
FC_eff = 0.45;
FC_P_d = 1; #kW/kg

#### INFO Turboprop ####
TURB_eff = 0.25;
TURB_P_d = 4; #kW/kg




# <h1>L'hybridation est elle vraiment utile pour l'avion à hydrogène? </h1>
<h2>Les avionneurs ont le choix: Combustion directe de l'hydrogène pour l'utiliser dans les turbofan classiques, ou conversion electrique grâce à une pile à combustible. </h2>



In [34]:
#@title

PART_TURB = 50;
PART_FC_W = widgets.IntSlider(
            layout= widgets.Layout(width='100%', height='30%'),
            value= 50, # default value
            description="Taux d'hybridation (%)",
            continuous_update=False,
            min= 0,
            max=100,
            step=1,
            style={'description_width': 'initial', 'handle_color': '#16a085'} 
    
    
        )








#@title

t_titre = "<h1>L'hybridation est elle vraiment utile pour l'avion à hydrogène? </h1>"
t_expl = "<h4>Les avionneurs ont le choix: Combustion directe de l'hydrogène pour l'utiliser dans les turbofan classiques, ou conversion electrique grâce à une pile à combustible. </h4>"
t_allechant = "<h4> Mais ils peuvent aussi opter pour un concept hybride, regroupant les forces de chacunes des technologies</h4>"
t_aller = "<h3> Faites bouger ce curseur pour determiner la pertinence d'un hybridation !</h3>"
t_onexplique = "<p>A gauche (0%) c'est combustion directe pure, comme les turboprops classique. A droite (100%) c'est purement pile à combustible</p>"

text_titre = widgets.HTML(value=t_titre)
text_expl = widgets.HTML(value=t_expl)
text_allechant = widgets.HTML(value=t_allechant)
text_aller = widgets.HTML(value=t_aller)
text_onexplique = widgets.HTML(value=t_onexplique)

vbox_text_test = widgets.VBox([text_titre, text_expl, text_allechant, text_aller, text_onexplique ])
display(vbox_text_test)

output2 = widgets.Output()

display(PART_FC_W,output2)




def on_value_change(change):
    with output2:
        
       
        output2.clear_output(wait=True)
        
        PART_FC = PART_FC_W.value/100
        PART_TURB = 1-PART_FC;
        mass_H2_FC = 60*Pmax*(Pdec*Tdec+Pmon*Tmon+Pcro*Tcro)/(H_E_d*FC_eff);


        if Pdec-PART_FC>0:
          Part_Turb_dec = Pdec-PART_FC;
        else:
          Part_Turb_dec = 0;

        if Pmon-PART_FC>0:
          Part_Turb_mon = Pmon-PART_FC;
        else:
          Part_Turb_mon = 0;

        if Pcro-PART_FC>0:
          Part_Turb_cro = Pcro-PART_FC;
        else:
          Part_Turb_cro = 0;

        mass_H2_TURB = 60*Pmax*(Part_Turb_dec*Tdec+Part_Turb_mon*Tmon+Part_Turb_cro*Tcro)/(H_E_d*TURB_eff);

        m_H2 = mass_H2_FC + mass_H2_TURB; #kg
        Vol_H2 = m_H2/H_d; #m3
        m_tank = m_H2/wt; #kg
        V_tank = Vol_H2/T_encomb; #m3
        L_tank = V_tank/(3.14*Diam_int**2/4); #m
        nb_pax_moins = int(4*L_tank);
        
        m_FC = PART_FC*Pmax/FC_P_d*1000; #kg
        m_TURB = PART_TURB*Pmax/TURB_P_d*1000; #kg
        m_prop = m_FC + m_TURB; #kg
        m_tank_prop = m_prop + m_tank; #kg
        
        opt = 4.727 ; #tonnes
        
        m_supp = m_tank_prop - opt*1000; #kg
        
        
        
        if 0<nb_pax_moins<=4:
            file = open("Tank_1.png", "rb")
        if 4<nb_pax_moins<=8:
            file = open("Tank_2.png", "rb")
        if 8<nb_pax_moins<=12:
            file = open("Tank_3.png", "rb")
        if 12<nb_pax_moins<=16:
            file = open("Tank_4.png", "rb")
        if 16<nb_pax_moins<=20:
            file = open("Tank_5.png", "rb")
        if 20<nb_pax_moins<=24:
            file = open("Tank_6.png", "rb")
        if 24<nb_pax_moins<=28:
            file = open("Tank_7.png", "rb")
        if 28<nb_pax_moins<=32:
            file = open("Tank_8.png", "rb")
            
        
        image = file.read()

        image_headline = widgets.Image(
                            value=image,
                            format='png',
                            width='300'
                        )

        label_headline = widgets.Label(
                            value='ATR 72 pour un vol de 3h',
                            style={'description_width': 'initial'}
                        )

        vbox_headline = widgets.VBox([image_headline, label_headline])
      
        t_0 = "<h3>"+"Masse reservoir: " + str(round(m_tank/1000,1))+" tonnes</h3>"
        t_1 = "<h3>"+"Masse propulsion: " + str(round(m_prop/1000,1))+" tonnes</h3>"
        t_2 = "<h2>"+"Masse totale: " + str(round(m_tank_prop/1000,1))+" tonnes</h2>"
        t_3 = "<p>"+"Soit " + str(int(m_supp))+" kg de plus que le point optimal, ce qui représente la masse de <b>" + str(round(m_supp/95))+" passagers</b></p>"
        t_4 = "<p>"+"De plus, le reservoir prend la place de <b>" + str(nb_pax_moins-11)+"</b> passagers supplémentaires par rapport au point optimal</p>"

        
        text_0 = widgets.HTML(value=t_0)
        text_1 = widgets.HTML(value=t_1)
        text_2 = widgets.HTML(value=t_2)
        text_3 = widgets.HTML(value=t_3)
        text_4 = widgets.HTML(value=t_4)
       

        vbox_text = widgets.VBox([text_0, text_1, text_2, text_3, text_4])

        page = widgets.HBox([vbox_headline, vbox_text])
        display(page)

PART_FC_W.observe(on_value_change, names='value')

t_ccl = "<h4>L'hybridation est essentielle pour que l'avion à hydrogène soit economiquement viable</h4>"
t_details = "<p>La pile à combustible possède un excellent rendement ce qui permet d'avoir des reservoirs plus petits, mais elle est très lourde. Toutefois, si elle est dimensionnée pour les phases de basses puissances (croisière) elle devient intéressante. Dans les phases de hautes puissances comme le decollage et la montée, le turbogenerateur, qui lui est leger mais a un mauvais rendement, vient completer ce manque de puissance.</p>"

text_ccl = widgets.HTML(value=t_ccl)
text_details = widgets.HTML(value=t_details)

vbox_text_test = widgets.VBox([text_ccl, text_details ])
display(vbox_text_test)








IntSlider(value=50, continuous_update=False, description="Taux d'hybridation (%)", layout=Layout(height='30%',…

Output()